# Hydro period calculation

- Given a series of land-water mask sentinel-1 RTC images, the code computes Hydroperiod
- Hydroperiod = no. of times a pixel appeared as water in mask / total number of masks

In [ ]:
import numpy as np
from osgeo import gdal,ogr, gdalconst
import subprocess
import glob
import os
from datetime import datetime
import matplotlib.pyplot as plt
import utm
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject
from skimage.filters import threshold_otsu

In [ ]:
def compute_hydroperiod(time_series_paths):
    hydroperiods = []
    
    for image_path in time_series_paths:
        with rasterio.open(image_path) as src:
            land_water_mask = src.read(1)
            hydroperiods.append(land_water_mask)
    
    # Sum the hydroperiods across the time series
    aggregated_hydroperiod = np.sum(hydroperiods, axis=0)
    # Normalize hydroperiod values (optional)
    hydroperiod = aggregated_hydroperiod / len(hydroperiods);
    hydroperiod[hydroperiod < 0] = -9999
    
    return hydroperiod

# Example usage
year = '2016'
LW_mask_dir = '/Users/bvarugu/Documents/Belomonte/Xingu_68_604_599_merged/VV_tifs/subset_tifs'
time_series_paths = sorted(glob.glob(LW_mask_dir+'/LW_mask__{}*.tif'.format(year)));
print(time_series_paths)
hydroperiod_2016 = compute_hydroperiod(time_series_paths);





In [ ]:
sample_rtc_file = sorted(glob.glob(os.path.join(LW_mask_dir, f'subset_{year}*_VV.tif')))[0]
with rasterio.open(sample_rtc_file) as src_rtc:
    rtc_profile = src_rtc.profile
    rtc_crs = src_rtc.crs
    rtc_transform = src_rtc.transform
    rtc_width = src_rtc.width
    rtc_height = src_rtc.height

def save_hydroperiod(hydroperiod, transform, crs, profile, output_path):
    profile.update(dtype=rasterio.float32, count=1, compress='lzw', transform=transform)
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(hydroperiod.astype(rasterio.float32), 1);
        dst.crs = crs
output_file = 'Hydroperiod_{}.tif'.format(year)
save_hydroperiod(hydroperiod_2016, rtc_transform, rtc_crs, rtc_profile, output_file)